### Use ``Open Route Service API`` to retrieve distance matrix
#### Quota 500 per day, approx 1 year of merged rental-place dataset per day. 
#### Specify the ``year`` of rental and place dataset before running 

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
import openrouteservice as ors
from add_distance import add_distance_time

In [2]:
PLACE_COLS = ['Place_Names', 'latitude', 'longitude', 'year', 'place_type', 'SA2_CODE']
place_df = pd.read_csv('../../data/curated/features_of_interst/place_all_with_sa2.csv')[PLACE_COLS]
print(place_df.shape)

(70444, 6)


### Alter the place and property dataframes for specific year (2013, 2014, ... 2022)

In [3]:
place_df = place_df[place_df['year'] == 2014]
place_df = place_df.drop_duplicates(subset=['Place_Names', 'latitude'])
print(place_df.shape)
place_df.head(10)

(7028, 6)


,Place_Names,latitude,longitude,year,place_type,SA2_CODE
2295,TAHARA BRIDGE RESERVE PICNIC AREA,-37.66568,141.66459,2014,park,217011420
2296,CULGOA RECREATION RESERVE,-35.71682,143.10101,2014,park,215031400
2297,CUMBERLAND,-38.57263,143.94681,2014,park,203031049
2298,WAREENA PARK,-36.35911,146.30977,2014,park,204021066
2299,ENTERPRIZE PARK,-37.82000,144.95973,2014,park,206041505
2300,RYANS RESERVE,-37.66807,145.15741,2014,park,209031215
2301,FERNTREE GULLY PICNIC GROUND,-37.89088,145.31692,2014,park,211051282
2302,LORRAINE KOVACS RESERVE,-37.97763,145.25403,2014,park,212021454
2303,RAY BASTIN RESERVE,-38.03359,145.30791,2014,park,212021456
2304,NEVILLE HAMILTON RESERVE,-38.02612,145.34533,2014,park,212021293


In [4]:
PROPERTY_COLS = ['address', 'latitude', 'longitude', 'nbed', 'nbath', 'ncar', 'weekly_rent', 'type', 'postcode', 'year', 'month', 'residence_type', 'SA2_CODE']
property_df = pd.read_csv('../../data/curated/property_all_with_SA2/2014_property_with_SA2.csv')[PROPERTY_COLS]
# 2013 example
print(property_df.shape)
property_df.head(10)

(12489, 13)


,address,latitude,longitude,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE
0,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027
1,"36 ALBERT STREET, CHEWTON",-37.093028,144.259469,3.0,1.0,1,320,House,3451,2014,8,House,202021027
2,"14 PITMAN STREET, CHEWTON",-37.083013,144.266589,4.0,1.0,1,345,House,3451,2014,1,House,202021027
3,"52 FRYERS ROAD, CHEWTON",-37.087738,144.257555,3.0,1.0,2,420,House,3451,2014,2,House,202021027
4,"114 FRYERS ROAD, CHEWTON",-37.094488,144.255400,2.0,1.0,3,260,House,3451,2014,9,House,202021027
5,"57 ARCHERS ROAD, CHEWTON",-37.094426,144.259118,3.0,1.0,2,395,House,3451,2014,5,House,202021027
6,"173 MAIN ROAD, CHEWTON",-37.080952,144.259378,2.0,1.0,0,270,House,3451,2014,10,House,202021027
7,"732A PYRENEES HIGHWAY, CHEWTON",-37.083670,144.272154,2.0,1.0,1,205,House,3451,2014,7,House,202021027
8,"1/20 NICKLESS STREET, CHILTERN",-36.154388,146.603242,2.0,1.0,1,200,House,3683,2014,1,House,204031070
9,"2/20 NICKLESS STREET, CHILTERN",-36.154038,146.602977,2.0,1.0,2,190,Rental_residential,3683,2014,3,Other,204031070


In [5]:
merged_df = property_df.merge(place_df, how='inner', on='SA2_CODE')
print(merged_df.shape)
print(f"Distinct sa2 codes = {merged_df['SA2_CODE'].nunique()}")
merged_df = merged_df.rename(columns={'latitude_x': 'latitude_ori', 'longitude_x': 'longitude_ori', 'latitude_y': 'latitude_des', 'longitude_y': 'longitude_des', 'year_x': 'year'})
merged_df.drop('year_y', axis=1, inplace=True)
merged_df.head(15)

(195960, 18)
Distinct sa2 codes = 483


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,OLD DIAMOND HILL RESERVE,-37.08179,144.20126,park
1,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CASTLEMAINE,-37.06316,144.21379,station
2,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CHEWTON,-37.08485,144.25848,station
3,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,WINTERS FLAT PRIMARY SCHOOL,-37.07016,144.20749,primary
4,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CASTLEMAINE NORTH PRIMARY SCHOOL,-37.05783,144.21899,primary
5,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CASTLEMAINE PRIMARY SCHOOL,-37.06660,144.22233,primary
6,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CHEWTON PRIMARY SCHOOL,-37.08233,144.26116,primary
7,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,CAMPBELLS CREEK PRIMARY SCHOOL,-37.09293,144.20466,primary
8,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,ST MARYS SCHOOL,-37.06396,144.22096,primary
9,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,10,House,202021027,Castlemaine High School,-37.05054,144.22750,secondary


#### Specify the ``client`` and ``year`` here

In [6]:
client = ors.Client('5b3ce3597851110001cf6248ce6c95ac96814219a4c3a7741f323b73') # Phikho-caz's api key 2500
#client = ors.Client('5b3ce3597851110001cf6248d99385fa95fc4e388d4e970e7e82a967') # Philly's api key 500
#client = ors.Client('5b3ce3597851110001cf6248e33707f5c3914f39b4671bac87646ac7') # Phikho's api key 500
#client = ors.Client('5b3ce3597851110001cf6248e33390389e0b4e0292fb3dcd2dec286b') # Phikhocaz's api key 500
#client = ors.Client('5b3ce3597851110001cf62482d289033372149f7b6419d839d18a3dc') # gracelovesyah api key 500 
#client = ors.Client('5b3ce3597851110001cf62488fb71274f7ac4b2ca2c679bedd963683') # gracelovesyah1 api key 500
#client = ors.Client('5b3ce3597851110001cf6248768e34108776435cabbcffaf2fb63a60') # kakakkaa api key 500
#client = ors.Client('5b3ce3597851110001cf624851a53f74cc8e4c6c9e751bb8aee27e05') # hyunjinp api key 500
#client = ors.Client('5b3ce3597851110001cf62488fd6bd483e964233809653e71984440a') # jonghop api key 500

In [7]:
added_distance_merged_df = add_distance_time(merged_df, 2014, client, 'saving') # specify the year here, client id, and mode of running (testing or saving)
#added_distance_merged_df = add_distance_time(subset, 2013)

Executing SA2 Code 202021027
Subset size = 476, Places count = 14, Property count = 34
To Place Distance Grand List, Normal Branch, length = 476
Executing SA2 Code 204031070
Subset size = 144, Places count = 9, Property count = 16
To Place Distance Grand List, Normal Branch, length = 620
Executing SA2 Code 210021235
Subset size = 120, Places count = 10, Property count = 12
To Place Distance Grand List, Normal Branch, length = 740
Executing SA2 Code 211051275
Subset size = 39, Places count = 3, Property count = 13
To Place Distance Grand List, Normal Branch, length = 779
Executing SA2 Code 209031212
Subset size = 598, Places count = 26, Property count = 23
To Place Distance Grand List, Normal Branch, length = 1377
Executing SA2 Code 211051276
Subset size = 440, Places count = 22, Property count = 20
To Place Distance Grand List, Normal Branch, length = 1817
Executing SA2 Code 205041094
Subset size = 1326, Places count = 39, Property count = 34
To Place Distance Grand List, Normal Branch

In [8]:
added_distance_merged_df.head(20)

,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,...,longitude_des,place_type,dist_to_place_M,dist_to_place_KM,time_to_place_S,time_to_place_MIN,dist_to_cbd_M,dist_to_cbd_KM,time_to_cbd_S,time_to_cbd_MIN
0,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.20126,park,882.99,0.88299,72.67,1.211167,131806.61,131.80661,5768.88,96.148000
1,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.21379,station,3676.80,3.67680,317.36,5.289333,131806.61,131.80661,5768.88,96.148000
2,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.25848,station,7601.72,7.60172,578.52,9.642000,131806.61,131.80661,5768.88,96.148000
3,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.20749,primary,3109.97,3.10997,362.66,6.044333,131806.61,131.80661,5768.88,96.148000
4,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.21899,primary,4110.13,4.11013,285.46,4.757667,131806.61,131.80661,5768.88,96.148000
5,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.22233,primary,3628.61,3.62861,284.06,4.734333,131806.61,131.80661,5768.88,96.148000
6,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.26116,primary,7660.90,7.66090,585.33,9.755500,131806.61,131.80661,5768.88,96.148000
7,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.20466,primary,535.74,0.53574,40.00,0.666667,131806.61,131.80661,5768.88,96.148000
8,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.22096,primary,3721.12,3.72112,320.93,5.348833,131806.61,131.80661,5768.88,96.148000
9,"1/85 MAIN ROAD, CHEWTON",-37.088043,144.204874,1.0,1.0,1,195,House,3451,2014,...,144.22750,secondary,5898.49,5.89849,463.34,7.722333,131806.61,131.80661,5768.88,96.148000
